# Lectura y balanceo de datos

In [1]:
#Paquetes para lectura y recuento
from sklearn.datasets import load_breast_cancer
import pandas as pd

#Primero importamos los datos que vamos a leer
cancer_data = load_breast_cancer()

#Pasamos los datos a un DataFrame (datos en pandas)
data = pd.DataFrame(cancer_data['data'], columns = cancer_data['feature_names'])
target = pd.DataFrame(cancer_data['target'],columns = ['Tumor Type'])

#Si quisieramos coger solo unas características
#data = data[['mean radius', 'mean texture']] 
#data

In [3]:
#Importamos los paquetes de sobremuestreo
from imblearn.over_sampling import SMOTE

#SMOTE
smote = SMOTE()

#Generación de nuevas muestras sintéticas
dataSmote, targetSmote = smote.fit_resample(data,target)

#Representación
dataSmote

# Inicialización de modelos

## Obtención de los conjuntos de evaluación y entrenamiento

In [4]:
from sklearn.model_selection import train_test_split
dataTrain, dataTest, targetTrain, targetTest = train_test_split(data,target, random_state = 0)

## Hiperparametrización del árbol de decisión

In [5]:
#Paquete
from sklearn.tree import DecisionTreeClassifier

#Inicialización.
modelDTS = DecisionTreeClassifier()

#Entrenamiento
modelDTS.fit(dataTrain, targetTrain)

#Predicción 
targetPredDTS = modelDTS.predict(dataTest)

In [17]:
#Importamos el paquete de la búsqueda Grid
from sklearn.model_selection import GridSearchCV

#Declaramos las variables que van a hiperparametrizarse
param_grid = [{'criterion': ['gini','entropy'],
              'max_depth': [None, 50, 100, 200, 500],
              'min_samples_split': [2,3,4]}]

#Inicializamos la búsqueda grid
grid = GridSearchCV(modelDTS, param_grid, cv = 5, n_jobs = -1, scoring = 'precision')

#Entrenamos con los datos de entreno...
grid.fit(dataTrain,targetTrain)

#Pintamos los resultados:
print("Mejores parámetros: ")
print(grid.best_params_)
print()
print("Puntuaciones: ")
print(grid.cv_results_['mean_test_score'])

In [8]:
targetPredGrid = grid.predict(dataTest)

In [18]:
#Paquetes
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#Obtención de la matriz
arrayDTS = confusion_matrix(targetPredDTS, targetTest)
arrayGrid = confusion_matrix(targetPredGrid, targetTest)
df_cmDTS = pd.DataFrame(arrayDTS)
df_cmGrid = pd.DataFrame(arrayGrid)

#Representación de la matriz de confusión
sn.set(font_scale=1)
plt.title('Árbol por defecto')
plt.show(sn.heatmap(df_cmDTS, annot=True, annot_kws={"size": 16}, cmap = 'Reds')) 
plt.title('Árbol hiperparametrizado')
plt.show(sn.heatmap(df_cmGrid, annot=True, annot_kws={"size": 16}, cmap = 'Reds')) 

In [19]:
#Paquete
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print('Exactitud árbol por defecto: ', accuracy_score(targetPredDTS,targetTest))
print('Exactitud árbol hiperparametrizado: ', accuracy_score(targetPredGrid,targetTest))
print('Memoria árbol por defecto: ', recall_score(targetPredDTS,targetTest))
print('Memoria árbol hiperparametrizado: ', recall_score(targetPredGrid,targetTest))
print('Precisión árbol por defecto: ', precision_score(targetPredDTS,targetTest))
print('Precisión árbol hiperparametrizado: ', precision_score(targetPredGrid,targetTest))
print('Puntuación F1 árbol por defecto: ', f1_score(targetPredDTS,targetTest))
print('Puntuación F1 árbol hiperparametrizado: ', f1_score(targetPredGrid,targetTest))

In [20]:
#Paquete
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#Obtención de las probabilidades de los datos de entrada
targetPredProbaDTS = modelDTS.predict_proba(dataTest)
targetPredProbaGrid = grid.predict_proba(dataTest)

#Obtención curva ROC
memDTS, unomenosespecDTS, umbralDTS = roc_curve(targetTest, targetPredProbaDTS[:,1])
memGrid, unomenosespecGrid, umbralGrid = roc_curve(targetTest, targetPredProbaGrid[:,1])

#Pintar Curva ROC
plt.title('ROC árboles')
plt.plot(memDTS, unomenosespecDTS,linestyle = ':')
plt.plot(memGrid, unomenosespecGrid, linestyle = '-.')
plt.plot([0,1],[0,1], linestyle = '--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.xlabel('1 - especificidad')
plt.ylabel('memoria')
plt.show()

print('Área por debajo de la curva del árbol por defecto: ', roc_auc_score(targetTest,targetPredProbaDTS[:,1]))
print('Área por debajo de la curva del árbol hiperparametrizado: ', roc_auc_score(targetTest,targetPredProbaGrid[:,1]))